In [139]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import pinecone
from pinecone import Pinecone, ServerlessSpec
import os

In [131]:
# loading pdf
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [132]:
# Creating text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [133]:
# Creating embeddings
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [134]:
extracted_data = load_pdf("../../End-to-end-Medical-Chatbot-using-Llama2/data")
text_chunks = text_split(extracted_data)
embeddings = download_hugging_face_embeddings()

In [135]:
query_result = embeddings.embed_query("medical reports")
print("Length", len(query_result))

Length 384


In [136]:
# Create pinecone API key and keep it in a environment variable
PINECONE_API_KEY = "pcsk_21V6u3_2B6jkUTmiQjaKtonBy2ncmv1yRhFdZqN5rvGwW7TdV6GCkEdT4MY2CtPCqn5GGu"
PINECONE_API_ENV = "chatbot1"
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [141]:
# Connect to pinecone and create index
pc = Pinecone(
    api_key=os.environ["PINECONE_API_KEY"]
)

index_name = "medical-chatbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [142]:
# Load the text_chunks to pinecone index
docsearch = LangChainPinecone.from_documents(text_chunks, embeddings, index_name=index_name)

In [143]:
query = "What is acne"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(metadata={'page': 39.0, 'page_label': '40', 'source': '..\\..\\End-to-end-Medical-Chatbot-using-Llama2\\data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'), Document(metadata={'page': 37.0, 'page_label': '38', 'source': '..\\..\\End-to-end-Medical-Chatbot-using-Llama2\\data\\Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is\nthe most common skin disease. It affects nearly 17 million\npeople in the United States. While acne can arise at any'), Document(metadata={'page': 38.0, 'page_label': '39', 'source': '..\\..\\End-to-end-Medical-Chatbot-using-Llama2\\data\\Medical

In [144]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [145]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [146]:
# Download llam model and keep it in model folder
llm=CTransformers(model="../../End-to-end-Medical-Chatbot-using-Llama2/model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [147]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [148]:
def getResponse(query):
    result=qa({"query": query})
    print("Response : ", result["result"])
    return result["result"]

In [ ]:
# Backend API using python flask
from flask import Flask, jsonify, request
from flask_cors import CORS


# Create a Flask app
app = Flask(__name__)
CORS(app)


@app.route('/chatbot', methods=['POST'])
def chatbot():
    print("called this function chatbot")
    data = request.get_json()  # Get the JSON data
    print(data)
    query = data.get('query')
    result=getResponse(query)
    return jsonify({"message": result})

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Jan/2025 20:44:49] "OPTIONS /chatbot HTTP/1.1" 200 -


called this function chatbot
{'query': 'symptoms of fever'}


127.0.0.1 - - [26/Jan/2025 20:46:04] "POST /chatbot HTTP/1.1" 200 -


Response :  The symptoms of acute lymphangitis are a high fever (usually between 38°–40°C), muscle aches, headache, chills, loss of appetite, and intermittent fevers.


127.0.0.1 - - [26/Jan/2025 20:46:29] "OPTIONS /chatbot HTTP/1.1" 200 -


called this function chatbot
{'query': 'suggest medecine for fever'}


127.0.0.1 - - [26/Jan/2025 20:48:21] "POST /chatbot HTTP/1.1" 200 -


Response :  For fever, acetaminophen (Tylenol) is usually recommended as it can help reduce both the temperature and pain associated with the infection. However, if the fever is very high (over 103°F or 39.4°C), aspirin may be given instead of acetaminophen as it can help reduce the fever more quickly. It's important to consult a doctor before giving any medication to a patient, especially if they have a history of allergies or other medical conditions.


In [151]:
from IPython.core.display import HTML


html_code = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Chatbot with Logs</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f0f0f0;
        }

        .chat-container {
            width: 80%;
            max-width: 600px;
            margin: 50px auto;
            padding: 20px;
            background-color: white;
            border-radius: 10px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
        }

        .chatbox {
            width: 100%;
            padding: 10px;
            margin: 10px 0;
            border: 1px solid #ddd;
            border-radius: 5px;
            box-sizing: border-box;
        }

        .chat-history {
            margin-bottom: 20px;
            max-height: 300px;
            overflow-y: auto;
        }

        .chat-message {
            margin-bottom: 10px;
        }

        .message-user {
            text-align: right;
            color: blue;
        }

        .message-bot {
            text-align: left;
            color: green;
        }

        button {
            width: 100%;
            padding: 10px;
            background-color: #4CAF50;
            color: white;
            border: none;
            border-radius: 5px;
            cursor: pointer;
        }

        button:hover {
            background-color: #45a049;
        }

        /* Logs Section Style */
        .logs-container {
            margin-top: 20px;
            padding: 10px;
            background-color: #f4f4f4;
            border-radius: 10px;
            border: 1px solid #ddd;
            max-height: 200px;
            overflow-y: auto;
            font-family: monospace;
            font-size: 14px;
            white-space: pre-wrap;
            word-wrap: break-word;
        }
    </style>
</head>
<body>

<div class="chat-container">
    <div class="chat-history" id="chat-history"></div>
    <textarea class="chatbox" id="question" placeholder="Ask a question..."></textarea>
    <button id="click_button" onclick="sendMessage()">Send</button>
</div>

<!--
<div class="logs-container" id="logs-container">
    Logs will be displayed here.
</div> -->

<script>
    // Ensure that the logs container exists before running the logging function
    const logsContainer = document.getElementById("logs-container");

    if (!logsContainer) {
        console.error("Logs container not found!");
    }

    async function sendMessage() {
        const question = document.getElementById("question").value;
        const chatHistory = document.getElementById("chat-history");

        if (!question.trim()) return;

        // Disable the text box and button
        document.getElementById('question').disabled = true;
        document.getElementById('click_button').disabled = true;

        // Display user's question
        const userMessage = document.createElement("div");
        userMessage.classList.add("chat-message", "message-user");
        userMessage.innerText = `You: ${question}`;
        chatHistory.appendChild(userMessage);
        
        // Log user message if logs container is available
        if (logsContainer) {
            logMessage(`User: ${question}`);
        }

        // Send question to the backend API
        try {
            const response = await fetch("http://127.0.0.1:5000/chatbot", {
                method: "POST",
                headers: {
                    "Content-Type": "application/json"
                },
                body: JSON.stringify({ query: question })
            });


            if (!response.ok) {
                throw new Error("Network response was not ok");
            }

            const data = await response.json();

            // Display bot's response
            const botMessage = document.createElement("div");
            botMessage.classList.add("chat-message", "message-bot");
            botMessage.innerText = `Bot: ${data.message}`;
            chatHistory.appendChild(botMessage);

            // Log bot response if logs container is available
            if (logsContainer) {
                logMessage(`Bot: ${data.message}`);
            }

            setTimeout(() => {
                document.getElementById('question').disabled = false;
                document.getElementById('click_button').disabled = false;
            }, 2000);
        } catch (error) {
            console.error('Error:', error);
            if (logsContainer) {
                logMessage(`Error: ${error.message}`);
            }
        }

        // Scroll to the bottom of the chat
        chatHistory.scrollTop = chatHistory.scrollHeight;

        // Clear the question input
        document.getElementById("question").value = '';
    }

    function logMessage(message) {
        if (!logsContainer) {
            console.error("Logs container not found!");
            return;
        }
        
        const logMessageElement = document.createElement("div");
        logMessageElement.innerText = message;
        logsContainer.appendChild(logMessageElement);

        // Scroll to the bottom of logs
        logsContainer.scrollTop = logsContainer.scrollHeight;
    }
</script>

</body>
</html>
"""

# Display the HTML
HTML(html_code)